In [3]:
import numpy as np
import metpy.calc as mpc
import metpy.constants as mconst
import xarray as xr
import scipy.ndimage as ndimage
import pandas as pd
import datetime as dt

In [25]:
### SCRIPT TO TRANSLATE DATA FROM SCAM NETCDF FILE TO SAS NETCDF FILE ####

### Constants

p0 = mconst.pot_temp_ref_press.magnitude
cp_air = mconst.dry_air_spec_heat_press.magnitude # Specific heat for dry air
r_gas = mconst.dry_air_gas_constant.magnitude   # Specific gas constant for dry air
r_cp = r_gas/cp_air    # r/cp
grav = mconst.earth_gravity.magnitude       # Gravity ave.

###### IOP file(s) info. ######

tlocal_offset_sec = -6*3600 # Hours behind Z (->seconds/hour)

scam_file_nums = ['104d'] # 'a'=L16, ''=L32
scam_run_names = ['CAM6']

scam_file_pre = "history/FSCAM.T42_T42.SAS."
scam_file_suf = ".cam.h0.2013-06-10-39600"

#------- Input/Output Files------------#
scam_files_in = [scam_file_pre+x+scam_file_suf for x in scam_file_nums]
sas_file_out = scam_files_in[0]+'_SAS_format.nc'


### READ IN NETCDF FILE/SET UP OUTPUT FILE ####
scam_in = xr.open_dataset(scam_files_in[0]+'.nc',engine='netcdf4')
#scam_sas =  xr.open_dataset(scam_files_in[0]+'_sas.format.nc')
#scam_in.rename_vars({'T':'RICHARD'})

scam_in = scam_in.squeeze() # Get rid dim=1 lat and lon

#------------ MakeDataSet copy --------------#
scam_out = scam_in.copy(deep=True)

#------ Dict of data variables ------------#
scam_vars  = scam_out.data_vars 

# Array of all 'data' variable names
scam_vars = list(scam_vars.keys())

#------- Grab SCAM Variable Names -------#
%run scam_nc2sas_vars # Run py file
sas_vars = list(var_r_df.index)

#------------ DROP all SCAM_out data variables  -------------#


scam_out = scam_out.drop_vars(scam_vars)
scam_out = scam_out.drop_vars(['lat','lon']) # Remov x,y coords


#---------- Straight copy/scale/rename of variables ----------#

print("#------- File Variable Writes ------#")
print('')

#------- Time Manipulation (seconds from loca midnight annoying)---------#

time_sas = scam_out.time.dt.second \
                +60*scam_out.time.dt.minute \
                +3600*scam_out.time.dt.hour \
                +86400*scam_out.time.dt.day


#------- Up Front Constants (just the values)  ---------#
scam_out['latitude'] = scam_in.lat.values    
scam_out['longitude'] = scam_in.lon.values   
        
# OK to drop lat/lon from scam_in now
#scam_in = scam_in.drop_vars(['lat','lon']) # Remove x,y coords




##############################
#---- LOOP ALL VARIABLES ----#
##############################

for vsas in sas_vars:
    vcam = var_r_df.loc[vsas]['SCAM Name']
    vname = var_r_df.loc[vsas]['Long Name']
    
    if vcam != '': # Direct transfer
 
        print('DIRECT  ++ Var (SAS/name/CAM) = ',vsas,' - ',vname,' - ',vcam)
    
# Rename (to SAS name) and scale vars to SAS values
        var = scam_in[vcam]*var_r_df.loc[vsas,'Scaling']


    
    #------ Derived Variables ------#

    if vcam == '':   # Derived or other variable
        print('DERVIED ++ Var (SAS/name/CAM) = ',vsas,' - ',vname,' - ',vcam)
            
# Vertical coords are 2D don't forget.        
        
        if vsas=='p':  pres,zhgt = vcoord_scam('mid')  ;  var=pres
        if vsas=='zu':  pres,zhgt = vcoord_scam('mid')  ;  var=zhgt
        if vsas=='zw':  
            pint,zint = vcoord_scam('int')  
            var=zint # Interface value that includes the surface value      
            
        
        if vsas in ['t','tv']: ## THETA/THETA_V ##
            pres,zhgt = vcoord_scam('mid')
            temp = scam_in.T 
            var = temp*(p0/pres)**r_cp 
            if vsas=='tv': var = var*(1+0.61*scam_in.T) 
            
## dq/dz / dtheta/dz for PBL depth

        if vsas in ['zi_q','zi_t']:
            var = vcoord_scam(vsas)
        
           
### Last constant variables and time
        if vsas=='time':
            var = time_sas-time_sas[0] # Scale by the first time (NCAR-LES has no time=0, SCAM does)
        if vsas in ['ts','cs']:  
            var = time_sas[0]-86400*scam_out.time.dt.day[0]+tlocal_offset_sec
            
   
## Add in general meta data.            
    var.attrs['description'] = var_r_df.loc[vsas,'Long Name']
    var.attrs['Units'] =  var_r_df.loc[vsas,'Units']
    var.attrs['z_loc'] =  1
    
## If an ilev array have to strip out surface value and assign z_loc=2

    var.attrs['z_loc'] = 2 if 'ilev' in var.dims else 1
#    var = var[1::] if 'ilev' in var.dims else var

#    scam_out[vsas]=var

    var = None
    
#------ Change Vertical and Time Dimension to to nz/nt

scam_out = scam_out.swap_dims({'time':'nt','lev':'nz'})

# Reverse arrays in the vertical 
print(scam_out.nz)
scam_out = scam_out.reindex(nz=scam_out.nz[::-1])
print(scam_out.nz)



#---------------- Write Out nc File -----------------#

scam_out.to_netcdf(sas_file_out)
        
print('-----------')
print('### END ###')
        
## JUNK


#        vsas0 = var_r_df.loc[var_r_df['SCAM Name'] == vcam] # Find index (SAS) variable name
#        vsas = vsas0.index # Get variable as the 'index' 
  
#        vname = var_r_df.loc[vsas]['Long Name']
#        vsas = vsas.map(str).values ; vsas = vsas[0] # Select first value to give single value - ridiculous
#        vname = vname.map(str).values ; vname = vname[0] # Same long-name
    

#------- File Variable Writes ------#

DERVIED ++ Var (SAS/name/CAM) =  ts  -  Simulation start time  -  
DERVIED ++ Var (SAS/name/CAM) =  cs  -  Chemistry start time  -  
DERVIED ++ Var (SAS/name/CAM) =  time  -  Time  -  
DERVIED ++ Var (SAS/name/CAM) =  zu  -  Height of grid at grid center  -  
DERVIED ++ Var (SAS/name/CAM) =  zw  -  Height of grid at grid upper-face  -  
DERVIED ++ Var (SAS/name/CAM) =  p  -  Pressure  -  
DIRECT  ++ Var (SAS/name/CAM) =  wqsfc  -  Surface water vapor specific humidity flux  -  LHFLX
DIRECT  ++ Var (SAS/name/CAM) =  wtsfc  -  Surface potential temperature flux  -  SHFLX
DERVIED ++ Var (SAS/name/CAM) =  t  -  Potential Temperature  -  
DERVIED ++ Var (SAS/name/CAM) =  tv  -  Virtual Potential Temperature  -  
DIRECT  ++ Var (SAS/name/CAM) =  q  -  Specific Humidity  -  Q
DERVIED ++ Var (SAS/name/CAM) =  zi_t  -  Height of maximum potential temperature gradient  -  
DERVIED ++ Var (SAS/name/CAM) =  zi_q  -  Height of maximum specific humidity gradien